# Library

In [2]:
import pandas as pd
import numpy as np
import re
import warnings

from konlpy.tag import Mecab
from gensim.models import FastText
from nltk.tokenize import sent_tokenize
from sqlalchemy import create_engine
from tqdm import tqdm_notebook

In [2]:
import pandas as pd
import numpy as np
import pickle as pk
import re
import tensorflow as tf

from gensim.models import FastText
from sentence_transformers import SentenceTransformer
from sqlalchemy import create_engine
from nltk import sent_tokenize
from sklearn.metrics.pairwise import cosine_similarity

In [4]:
# 경고무시를 위한 코드
warnings.filterwarnings('ignore')

# 데이터 로드

In [4]:
case_list_사기 = pd.read_csv('./00. Data/case_list_사기.csv')

In [5]:
case_list_사기.columns

Index(['Unnamed: 0', '판례일련번호', '사건명', '사건번호', '선고일자', '법원명', '사건종류명', '사건종류코드',
       '판결유형', '선고', '판례상세링크', '판시사항', '판결요지', '참조조문', '참조판례', '판례내용', 'X'],
      dtype='object')

# 전처리

## 문장 토큰화 전 전처리

In [6]:
case_list_사기['X'] = case_list_사기['판례내용']

### 판례내용 한자를 한글로 바꾸기

In [7]:
# 엑셀에서 한자 있는 행 찾음
if '甲' in case_list_사기['X'].iloc[357]:
    print('Yes')
else:
    print('No')

Yes


In [8]:
from hanja_hangul.src import hanjahangul
# 'X' 열에 대해 'hanjahangul.hanja_to_hangul_dueum' 함수 적용

def safe_hanja_to_hangul(text):
    try:
        return hanjahangul.hanja_to_hangul_dueum(text)
    except ValueError:
        # 오류 발생시 처리 방법 (예: 원본 텍스트 반환)
        return text

case_list_사기['X'] = case_list_사기['X'].apply(safe_hanja_to_hangul)
# 50초 소요

In [9]:
# 엑셀에서 한자 있는 행 찾음
if '甲' in case_list_사기['X'].iloc[357]:
    print('Yes')
else:
    print('No')

No


### '유】', '유]'로 자르기

In [10]:
count = 0
for i in range(len(case_list_사기)): # '【이    유】' 포함 행 세기
    if '유】' or '유]' in case_list_사기['X'][i]:
        count += 1
print(count) # 2262개 중 2262개 행에 있음

2262


In [11]:
for i in range(len(case_list_사기)): # '【이    유】'로 split 한 후, 이유 앞은 버리기
    if '【이    유】' in case_list_사기['X'][i]:
        case_list_사기['X'][i] = case_list_사기['X'][i].split('【이    유】')[1]

In [12]:
for i in range(len(case_list_사기)): # '[이    유]'로 split 한 후, 이유 앞은 버리기
    if '[이    유]' in case_list_사기['X'][i]:
        case_list_사기['X'][i] = case_list_사기['X'][i].split('[이    유]')[1]

In [13]:
case_list_사기['X'].head()

0      상고이유를 판단한다.  1. 불가벌적 사후행위에 대한 판단「유사수신행위의 규제에...
1      상고이유를 판단한다.   1. 피고인 1, 피고인 2, 피고인 3, 피고인 4,...
2      직권으로 판단한다.   1. 의료법 위반의 점에 관한 공소사실의 요지   피고인...
3      상고이유를 판단한다.  1. 사건의 개요와 쟁점   가. 의료법 위반의 점에 관...
4      1. 이 법원의 심판범위  원심은 배상신청인들의 배상신청을 모두 각하하였다. 소...
Name: X, dtype: object

### 정규표현식 사용하여 필요없는 문자 제거

In [14]:

def remove_text(text):

    court = r'대법원 \d{4}\. \d{1,2}\. \d{1,2}\. 선고 \S+ 판결' # 대법원 1992. 12. 22. 선고 92다14779 판결, 대법원 1999. 4. 27. 선고 99두202 판결
    number_pattern1 = r'제\d+[조항호]' # 제{}조 제{}항 제{}호
    number_pattern2 = r'\d{4}\. \d{1,2}\. \d{1,2}\.' # 날짜 제거
    text_pattern1 = r'\s[가-힣]\.' # 가., 나. / 했다. 이런것도 없애지 않기 위해 앞에 공백 있는 것만 지우기
    text_pattern2 = r'\n' # \n
    text_pattern3 = r'\([가-힣]\)' # (가), (나)..
    
    final_patten = r'[^가-힣.\s]' # 즉, 한글, 마침표, 공백을 제외한 모든 문자를 제거하는 작업

    modified_text = re.sub(court, '', text)
    modified_text = re.sub(number_pattern1, '', modified_text)
    modified_text = re.sub(number_pattern2, '', modified_text)
    modified_text = re.sub(text_pattern1, '', modified_text)
    modified_text = re.sub(text_pattern2, '', modified_text)
    modified_text = re.sub(text_pattern3, '', modified_text)
    modified_text = re.sub(final_patten, '', modified_text) 

    return modified_text

In [15]:
case_list_사기['X'] = case_list_사기['X'].apply(remove_text)

In [16]:
case_list_사기['X'].head()

0      상고이유를 판단한다.  . 불가벌적 사후행위에 대한 판단유사수신행위의 규제에 관...
1      상고이유를 판단한다.   . 피고인  피고인  피고인  피고인  피고인 에 대한...
2      직권으로 판단한다.   . 의료법 위반의 점에 관한 공소사실의 요지   피고인은...
3      상고이유를 판단한다.  . 사건의 개요와 쟁점   의료법 위반의 점에 관한 공소...
4      . 이 법원의 심판범위  원심은 배상신청인들의 배상신청을 모두 각하하였다. 소송...
Name: X, dtype: object

### 쉬운 단어로 대체

In [17]:
# 알기 쉬운 법령 정비기준 참고

replace_words = {}
replace_words['개임'] = '교체 임명'
replace_words['개피'] = '개봉'
replace_words['구거'] = '도랑'
replace_words['부의하는'] = '회의에 부치는'
replace_words['사위'] = '거짓'
replace_words['실사'] = '실제조사'
replace_words['자중'] = '무게'
replace_words['잔임기간'] = '남은 임기'
replace_words['전폐된'] = '모두 잃은'
replace_words['필증'] = '증명서'
replace_words['필하다'] = '마치다'
replace_words['필한'] = '마친'
replace_words['허위'] = '거짓'
replace_words['회무'] = '업무'
replace_words['파훼'] = '파기'
replace_words['아니하지 않을 수 없다'] = '아니하다'

In [18]:
for key in replace_words.keys():
    case_list_사기['X'] = case_list_사기['X'].str.replace(key, replace_words[key])

In [48]:
# case_list_사기.to_csv('case_list_사기.csv')